# Fusion Retrieval in Document Search

## Overview

This code implements a Fusion Retrieval system that combines vector-based similarity search with keyword-based BM25 retrieval. The approach aims to leverage the strengths of both methods to improve the overall quality and relevance of document retrieval.

## Motivation

Traditional retrieval methods often rely on either semantic understanding (vector-based) or keyword matching (BM25). Each approach has its strengths and weaknesses. Fusion retrieval aims to combine these methods to create a more robust and accurate retrieval system that can handle a wider range of queries effectively.

## Key Components

1. PDF processing and text chunking
2. Vector store creation using FAISS and OpenAI embeddings
3. BM25 index creation for keyword-based retrieval
4. Custom fusion retrieval function that combines both methods

## Method Details

### Document Preprocessing

1. The PDF is loaded and split into chunks using RecursiveCharacterTextSplitter.
2. Chunks are cleaned by replacing 't' with spaces (likely addressing a specific formatting issue).

### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store is created from these embeddings for efficient similarity search.

### BM25 Index Creation

1. A BM25 index is created from the same text chunks used for the vector store.
2. This allows for keyword-based retrieval alongside the vector-based method.

### Fusion Retrieval Function

The `fusion_retrieval` function is the core of this implementation:

1. It takes a query and performs both vector-based and BM25-based retrieval.
2. Scores from both methods are normalized to a common scale.
3. A weighted combination of these scores is computed (controlled by the `alpha` parameter).
4. Documents are ranked based on the combined scores, and the top-k results are returned.

## Benefits of this Approach

1. Improved Retrieval Quality: By combining semantic and keyword-based search, the system can capture both conceptual similarity and exact keyword matches.
2. Flexibility: The `alpha` parameter allows for adjusting the balance between vector and keyword search based on specific use cases or query types.
3. Robustness: The combined approach can handle a wider range of queries effectively, mitigating weaknesses of individual methods.
4. Customizability: The system can be easily adapted to use different vector stores or keyword-based retrieval methods.

## Conclusion

Fusion retrieval represents a powerful approach to document search that combines the strengths of semantic understanding and keyword matching. By leveraging both vector-based and BM25 retrieval methods, it offers a more comprehensive and flexible solution for information retrieval tasks. This approach has potential applications in various fields where both conceptual similarity and keyword relevance are important, such as academic research, legal document search, or general-purpose search engines.

<div style="text-align: center;">

<img src="../images/fusion_retrieval.svg" alt="Fusion Retrieval" style="width:100%; height:auto;">
</div>

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
#CELL-NO: 1
# Install required packages
#!uv pip install langchain langchain-openai langchain-community langchain-text-splitters faiss-cpu numpy python-dotenv rank-bm25 pymupdf

In [ ]:
#CELL-NO: 2
import os
import sys
from dotenv import load_dotenv
from langchain_community.docstore.document import Document

from typing import List
from rank_bm25 import BM25Okapi
import numpy as np


from utils.helper_functions import *
from utils.evaluate_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Define document path

In [ ]:
#CELL-NO: 3
path = "data/Understanding_Climate_Change.pdf"

### Encode the pdf to vector store and return split document from the step before to create BM25 instance

In [ ]:
#CELL-NO: 4
def encode_pdf_and_get_split_documents(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A tuple of (FAISS vector store, cleaned text documents).
    """
    from langchain_community.document_loaders import PyPDFLoader
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_openai import OpenAIEmbeddings
    from langchain_community.vectorstores import FAISS

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore, cleaned_texts

### Create vectorstore and get the chunked documents

In [ ]:
#CELL-NO: 5
vectorstore, cleaned_texts = encode_pdf_and_get_split_documents(path)

### Create a bm25 index for retrieving documents by keywords

In [ ]:
#CELL-NO: 6
from rank_bm25 import BM25Okapi

# Initalizing
corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]
print(tokenized_corpus)

bm25 = BM25Okapi(tokenized_corpus)

# Ranking of documents
query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

doc_top_n = bm25.get_top_n(tokenized_query, corpus, n=1)
print(doc_top_n)

In [ ]:
#CELL-NO: 7
#sparse embedding generation
def create_bm25_index(documents: List[Document]) -> BM25Okapi:
    """
    Create a BM25 index from the given documents.

    BM25 (Best Matching 25) is a ranking function used in information retrieval.
    It's based on the probabilistic retrieval framework and is an improvement over TF-IDF.

    Args:
    documents (List[Document]): List of documents to index.

    Returns:
    BM25Okapi: An index that can be used for BM25 scoring.
    """
    # Tokenize each document by splitting on whitespace
    # This is a simple approach and could be improved with more sophisticated tokenization
    tokenized_docs = [doc.page_content.split() for doc in documents]
    return BM25Okapi(tokenized_docs)

In [ ]:
#CELL-NO: 8
bm25 = create_bm25_index(cleaned_texts) # Create BM25 index from the cleaned texts (chunks)

### Define a function that retrieves both semantically and by keyword, normalizes the scores and gets the top k documents

In [ ]:
#CELL-NO: 9
def fusion_retrieval(vectorstore, bm25, query: str, k: int = 5, alpha: float = 0.5) -> List[Document]:
    """
    Perform fusion retrieval combining keyword-based (BM25) and vector-based search.

    Args:
    vectorstore (VectorStore): The vectorstore containing the documents.
    bm25 (BM25Okapi): Pre-computed BM25 index.
    query (str): The query string.
    k (int): The number of documents to retrieve.
    alpha (float): The weight for vector search scores (1-alpha will be the weight for BM25 scores).

    Returns:
    List[Document]: The top k documents based on the combined scores.
    """
    
    epsilon = 1e-8

    # Step 1: Get all documents from the vectorstore
    all_docs = vectorstore.similarity_search("", k=vectorstore.index.ntotal)

    # Step 2: Perform BM25 search
    bm25_scores = bm25.get_scores(query.split())

    # Step 3: Perform vector search
    vector_results = vectorstore.similarity_search_with_score(query, k=len(all_docs))
    
    # Step 4: Normalize scores
    vector_scores = np.array([score for _, score in vector_results])
    vector_scores = 1 - (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)

    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) -  np.min(bm25_scores) + epsilon)

    # Step 5: Combine scores
    combined_scores = alpha * vector_scores + (1 - alpha) * bm25_scores  

    # Step 6: Rank documents
    sorted_indices = np.argsort(combined_scores)[::-1]
    
    # Step 7: Return top k documents
    return [all_docs[i] for i in sorted_indices[:k]]

### Use Case example

In [ ]:
#CELL-NO: 10
# Query
query = "What are the impacts of climate change on the environment?"

# Perform fusion retrieval
top_docs = fusion_retrieval(vectorstore, bm25, query, k=5, alpha=0.5)
docs_content = [doc.page_content for doc in top_docs]
show_context(docs_content)

## Langchain Implementation

In [ ]:
#CELL-NO: 11
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_classic.retrievers.ensemble import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_classic.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

# Load PDF
loader = PyPDFLoader("data/Understanding_Climate_Change.pdf")
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50
)
splits = text_splitter.split_documents(documents)

# Dense retriever (semantic)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(splits, embeddings)
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Sparse retriever (keyword - BM25)
sparse_retriever = BM25Retriever.from_documents(splits)
sparse_retriever.k = 3

# Hybrid retriever
alpha:float = 0.5
hybrid_retriever = EnsembleRetriever(
    retrievers=[dense_retriever, sparse_retriever],
    weights=[alpha, 1.0 - alpha]
)

# LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Create RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=hybrid_retriever,
    return_source_documents=True
)

# Query with generation
query = "What is the main topic of this document?"
result = qa_chain.invoke({"query": query})

print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.page_content[:200]}...")

## LlamaIndex

In [ ]:
#CELL-NO: 12
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Configure LLM and embeddings
Settings.llm = Ollama(model="llama3.2", temperature=0)
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Load PDF
documents = SimpleDirectoryReader(
    input_files=["document.pdf"]
).load_data()

# Create index with chunking
index = VectorStoreIndex.from_documents(
    documents,
    node_parser=SentenceSplitter(
        chunk_size=512,
        chunk_overlap=50
    )
)

# Create query engine with hybrid retriever
query_engine = index.as_query_engine(
    similarity_top_k=3,
    vector_store_query_mode="hybrid",
    alpha=0.5
)

# Query with generation
response = query_engine.query("What is the main topic of this document?")

print("Answer:", response.response)
print("\nSources:")
for node in response.source_nodes:
    print(f"- {node.text[:200]}...")
    print(f"  Score: {node.score}")

## Direct ChromaDB Integration

- Pinecone
- Weviate

In [ ]:
#CELL-NO: 13
import chromadb
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.chat_models import ChatOllama

# Load PDF
loader = PyPDFLoader("data/Understanding_Climate_Change.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# Setup ChromaDB
client = chromadb.PersistentClient(path="./chroma_db")
collection = client.get_or_create_collection("pdf_docs")

# Embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Add documents
for idx, chunk in enumerate(chunks):
    collection.add(
        ids=[f"doc_{idx}"],
        documents=[chunk.page_content],
        embeddings=[model.encode(chunk.page_content).tolist()],
        metadatas=[{"page": chunk.metadata.get("page", 0)}]
    )

# Hybrid Query
query = "What are the main topics?"
query_emb = model.encode(query)

results = collection.query(
    query_embeddings=[query_emb.tolist()],
    n_results=5
)

# LLM Generation
llm = ChatOllama(model="llama3.2")
context = "\n\n".join(results['documents'][0])
prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
response = llm.invoke(prompt)

print("Answer:", response.content)

### RRF

In [ ]:
#CELL-NO: 14
#!uv pip install scikit-learn sentence_transformers

In [ ]:
#CELL-NO: 15
#TODO - assignment
def relative_score_fusion(scorings, k=60, weights=None):
    """RRF implementation"""
    if weights is None:
        weights = [1.0] * len(scorings)
    
    doc_scores = {}
    for scoring_idx, ranking in enumerate(scorings):
        weight = weights[scoring_idx]
        for score_position, doc_id in enumerate(scoring, start=1):
            rrf_score = weight * (1.0 / (k + score_position))
            doc_scores[doc_id] = doc_scores.get(doc_id, 0.0) + rrf_score
    
    return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)



In [ ]:
#CELL-NO: 16
def reciprocal_rank_fusion(rankings, k=60, weights=None):
    """RRF implementation"""
    if weights is None:
        weights = [1.0] * len(rankings)
    
    doc_scores = {}
    for ranking_idx, ranking in enumerate(rankings):
        weight = weights[ranking_idx]
        for rank_position, doc_id in enumerate(ranking, start=1):
            rrf_score = weight * (1.0 / (k + rank_position))
            doc_scores[doc_id] = doc_scores.get(doc_id, 0.0) + rrf_score
    
    return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)


In [ ]:
#CELL-NO: 15
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


# Sample documents
documents = [
    "Machine learning is a subset of artificial intelligence",
    "Deep learning uses neural networks with multiple layers",
    "Python is popular for data science and ML",
    "Natural language processing helps computers understand text",
    "Computer vision enables machines to interpret images"
]

# Query
query = "What is AI"

# ============================================
# DENSE SEARCH (Vector/Semantic)
# ============================================
dense_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = dense_model.encode(documents)
query_embedding = dense_model.encode(query)

# Calculate cosine similarity
similarities = np.dot(doc_embeddings, query_embedding) / (
    np.linalg.norm(doc_embeddings, axis=1) * np.linalg.norm(query_embedding)
)

# Get ranked document IDs (by similarity)
dense_ranking = np.argsort(similarities)[::-1].tolist()

# ============================================
# SPARSE SEARCH (Keyword/BM25-like)
# ============================================
tfidf = TfidfVectorizer()
doc_tfidf = tfidf.fit_transform(documents)
query_tfidf = tfidf.transform([query])

# Calculate TF-IDF scores
tfidf_scores = (doc_tfidf @ query_tfidf.T).toarray().flatten()

# Get ranked document IDs
sparse_ranking = np.argsort(tfidf_scores)[::-1].tolist()

# ============================================
# HYBRID SEARCH WITH RRF
# ============================================
hybrid_results = reciprocal_rank_fusion(
    rankings=[dense_ranking, sparse_ranking],
    k=60,
    weights=[2.0, 1.0]  # Dense search more important
)

# ============================================
# DISPLAY RESULTS
# ============================================
print("QUERY:", query)
print("\n" + "="*60)
print("DENSE SEARCH RANKING:")
for rank, doc_idx in enumerate(dense_ranking, 1):
    print(f"{rank}. [{doc_idx}] {documents[doc_idx]}")

print("\n" + "="*60)
print("SPARSE SEARCH RANKING:")
for rank, doc_idx in enumerate(sparse_ranking, 1):
    print(f"{rank}. [{doc_idx}] {documents[doc_idx]}")

print("\n" + "="*60)
print("HYBRID RRF RESULTS:")
print("="*60)
for doc_idx, score in hybrid_results[:3]:  # Top 3
    print(f"Score: {score:.6f}")
    print(f"Doc:   {documents[doc_idx]}")
    print()

### RSF

In [ ]:
#CELL-NO: 18
